<div style="text-align: right">  
Notebook by Sem Vijverberg (Adapted from Ted Veldkamps and Marthe Wens) <br>
</div>

## Drought risk: ENSO impact on drought hazard for Kenya


## Introduction

In the lecture, you were taught about ways to estimate meteorological, agricultural, and hydrological drought hazard. Either by using standardized indicators (SPI, SPEI, SSFI) or by fixed or variable threshold level methods (Q90). In this practical, you will quantify African drought using the SPI indicator and discover the impact of ENSO on Drought. 

Learning goals for this practicum are:
- Learn to apply standardized methods to represent meteorological and hydrological drought hazard;
- Learn to deal with multi-dimensional climate datasets.
- Learn to visualize and interpret the results from drought hazard calculation methods.

## Performing the exercise
To run any script in the code-boxes below use *shift+enter*. **action** denotes where you have to make changes to particular pieces of code. When a script is running this is indicated by the * on the left side of the window.

## Saving your results
See the word answer sheet in the main repository. Make sure you safe the requested outputs (figures, tables) and answers to questions here. Furthermore, save the codes that you have created and run during this exercise (right clicking the .ipynb and select download).

## Working in a Binder environment?
Binder is super helpfull in the sense that all Python dependencies will work. **But a Binder can be unstable.** After you have made changes to the code please copy and paste that into a local text file to ensure you can easily retrieve progress.

## 1. Importing Python packages

Before running any Python script (in an offline or online modus) it is necessary to import a number of Python packages that can help you with performing the calculations. 

**Action**: Perform the script in the code-boxes below by selecting them and press *shift+enter*.

In [ ]:
# Import working modules

import xarray as xr
import numpy as np
import scipy.stats as stats
import matplotlib
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
import inspect, os, sys

# Define paths
workingfolder = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
if workingfolder.split('/')[-1] != 'Answers':
    Notebooks_path = os.path.join(workingfolder, 'Notebooks')
else:
    Notebooks_path = os.path.join(workingfolder, '..', 'Notebooks')
    
# adding path of notebooks to sys.path
if Notebooks_path not in sys.path:
    sys.path.append(Notebooks_path)
import core_pp

matplotlib.rcParams['font.size'] = 18
matplotlib.rcParams['legend.fontsize'] = 'xx-large'

In the Data-folder a number of files have been prepared that serve as input for this exercise. First, open the "precip_file" and "evap_file" familiarize yourself with the data. 

**Action**: Make sure that path is correct. If needed, make changes to input_folder and input_file. Afterwards run the code by pressing *ctrl+enter*.

In [ ]:
inputfolder   = os.path.join(workingfolder,'..', 'Data')

# Name input file
precip_file   = os.path.join(inputfolder,'p_1979-2018_Africa.nc')
evap_file     = os.path.join(inputfolder,'PE_1979-2018_Africa.nc')
country_codes = os.path.join(inputfolder,'Africa_mask_Countries.nc')
sst_Pacific   = os.path.join(inputfolder,'sst_1979-2018_Trop_Pacific.nc')

## 2: Loading Precipitation and Potential Evaporation data

You will now start with loading and inspecting the input data within Python Jupyter Notebooks. You will do this in the following code-box. Press *ctrl+enter* to run the code.

**Action**: Load the data into the memory of this kernel.

In [ ]:
precip_m  = core_pp.import_ds_lazy(precip_file)
evap_m    = core_pp.import_ds_lazy(evap_file)

**Action**: Inspect the data set by plotting the climalogical rainfall and potential evaporation over Africa. 

In [ ]:
precip_m

### 2a: Inspecting the data
In python you can store data in different manners. Famous data types are Lists, Numpy arrays, dictionairies, or pandas Dataframe. Each data type has advantages and disadvantage. An xarray.DataArray is super handy for multi-dimensional datasets. It is like an excell sheets, but we can add as many dimensions as we want. For normal Excell sheets, we are limited to either the row or column dimensions, xarray can handle multiple. In this case, we see the time, latitude (imagine the y-axis) and longitude (imagine the x-axis) dimensions. We can also see that units are in Meters (per day), lets convert that to mm per day get a better feeling.

In [ ]:
precip = precip_m * 1000
precip.name = 'Precipitation [mm/day]'
precip.attrs['units'] = '[mm/day]'
evap   = evap_m * 1000
evap.name = 'Potential Evaporation [mm/day]'
evap.attrs['units'] = '[mm/day]'

In the cell below we create the most simple plot of the climatological rainfall over Africa. For a climatology, we at least need 30 years. This is true particularly for precipiation, some argue we need at least 50 years to get a good climatology.


In [ ]:
(precip).mean(dim='time').plot.contourf()

**Question 0:** Why do you think we need more years to get a robust climatology compared to other variables?

Using the Python Package Cartopy (ccrs), we can make nice spatial maps.

**Question 1:** Briefly explain what you can learn from figure 1.

In [ ]:
# Figure 1
# Use Cartopy Python package to plot a map and draw country lines
map_proj = ccrs.PlateCarree(central_longitude=10)

fig = plt.figure(figsize=(30,10) )

# plot precipitation
# The integer numbers for plt.subplot refer to (n_rows, n_cols, 'index of figure') 
ax1 = plt.subplot(1, 2, 1, projection=map_proj) 
colormap = plt.cm.Blues
precip.mean(dim='time').plot.contourf(ax=ax1, transform=ccrs.PlateCarree(), 
                             subplot_kws={'projection': map_proj},
                             cmap=colormap)
ax1.coastlines()
# plot evaporation
ax2 = plt.subplot(1, 2, 2, projection=map_proj)
colormap = plt.cm.coolwarm_r
evap.mean(dim='time').plot.contourf(ax=ax2, transform=ccrs.PlateCarree(), 
                             subplot_kws={'projection': map_proj},
                             cmap=colormap)
ax2.coastlines()


In [ ]:
# (precip + (evap)).mean(dim='time').plot.contourf()

### 2b: Retrieve data for Kenya
First we will focus on quantifying drought in Kenya. In order to retrieve only data from that country, we load the 'country_codes'.

**Action**: Load country_codes and select precipiation and evaporation of Kenya. 

In [ ]:
country = xr.open_dataset(country_codes).country_mask
land_sea_mask = country >= 1
map_proj = ccrs.PlateCarree(central_longitude=10)
colormap = plt.cm.gist_ncar_r
fig = plt.figure(figsize=(15,10) )

ax1 = plt.subplot(1, 1, 1, projection=map_proj)
country.plot(ax=ax1, transform=ccrs.PlateCarree(), 
                            subplot_kws={'projection': map_proj},
                            cmap=colormap,
                            vmin=0, vmax=country.max())

The *country* file contains an attribute called 'country_code'. The following cell will convert this information into a Python dictionairy. A python dictionairy consists of keys and items in the following format: {*key1* : *item1*}. In the following, the key refers to an abbreviation of the country nams and the item refers to the numbers in the country mask. 

We will focus the coming analysis on Kenia, which has the *key* KE. 

In [ ]:
c_list = country.attrs['country_code']
codes_dict = {}
for i in np.arange(0,len(c_list),2):
    codes_dict[str(c_list[i])] = int(c_list[i+1])
codes_dict['KE']

Thus we need to gridpoints corresponding to number 115 in the *country* xarray. We will make a boolean mask (consisting of only True and False), which will only be True at Kenya. We can use that mask to select the gridcells of the *precip* and *evap* xarray.

In [ ]:
Kenya_mask = country.values == 115
Kenya_mask = (('latitude', 'longitude'), Kenya_mask )
precip.coords['mask'] = Kenya_mask
pr_kenya = precip.where(precip.mask)


**Action**: Add the Kenya country mask to the evaporation dataset

In [ ]:
map_proj = ccrs.PlateCarree(central_longitude=10)

fig = plt.figure(figsize=(30,10) )

# plot precipitation
ax1 = plt.subplot(1, 2, 1, projection=map_proj)
colormap = plt.cm.Blues
pr_kenya.mean(dim='time').plot(ax=ax1, transform=ccrs.PlateCarree(), 
                             subplot_kws={'projection': map_proj},
                             cmap=colormap,
                             vmin=-3, vmax=3)
ax1.coastlines()
# plot evaporation
ax2 = plt.subplot(1, 2, 2, projection=map_proj)
colormap = plt.cm.RdBu
ev_kenya.mean(dim='time').plot(ax=ax2, transform=ccrs.PlateCarree(), 
                             subplot_kws={'projection': map_proj},
                             cmap=colormap,
                                vmin=-3, vmax=3)
ax2.coastlines()

## 3: SPI for a single gridcell in Kenya

## Steps to calculate SPI

1. SPI is bases on temporally aggregated values of precipitation (in mm). Common temporal aggregations are 1, 3 and 6 months, but their are no stricts rules. You can simply call it SPI-x, with x representing the amount of accumulated months.
2. We then fit a distribution (across years), often the gamma distribution is used. However, given the strong heterogeneity around the globe, this distribution is not always the best. Preferably one tests different distributions, and chooses the best one. For this exercise, we will stick to Gamma distribution.  
3. The fitted distribution gives us the probability that a value occurs, this probability is transformed to standard normal distribution to ease interpretation. Stagge et al. (2015) pointed out that we should remove all zeros when we do this transformation to ensure that the mean of the new standard normal distribution will be zero.   

Note, since we are working with daily data, we take aggregation over i-days, representing the monthly aggregations. i.e. 1, 3, 6, which are considered equivalent to 31, 91, 183 days.


### First get a single gridcell from Kenya

In [ ]:
# we need to do a technical conversion to retrieve only the gridcells of Kenya
np_pr_kenya = np.reshape(pr_kenya.values, (pr_kenya.time.size, -1))
mask_NanIsFalse   = ~np.isnan(np_pr_kenya) # where are values not NaN 
# select only where values not NaN (Kenya gridcells)
pr_kenya_v  = np_pr_kenya[mask_NanIsFalse].reshape( pr_kenya.time.size, -1) 
df_kenya = pd.DataFrame(pr_kenya_v, index=pr_kenya.time) # make a pandas dataframe

# single gridcell
i_gridcell = 0
df_gs = df_kenya[i_gridcell]

### 3 - step 1: Apply rolling sum

In [ ]:
SPI_aggr = 1
months = [1, 3, 6]
days = [31, 91, 183]
SPI_to_days = {months[i]:days[i] for i in range(len(months))}
SPI_window = SPI_to_days[SPI_aggr]

# the following line calculates a sum of window size {SPI_aggr}, using equal weights. 
df_kenya = df_gs.rolling(window=SPI_window, min_periods=1, center=True, axis=0).sum()

### 3 - step 2: Fit distribution for single gridcell and single day of year

In [ ]:
# Because we are calculating SPI-{n} for each day of the year by using the rolling sum approach,
# we only need to select a single day to represent a n-month aggregation. 

# to select the SPI for month m (with 1 = jan and 12 = Dec):
m = 4
dates = df_gs.index
doy = int(np.mean(dates.dayofyear[dates.month == m])) # central dayofyear for the given month {m}

df_gs_m = df_gs[dates.dayofyear == doy]

# Fit Gamma distibution
# note floc avoids loc being fitted, see Stagge et al. 2015
a, loc, scale = stats.gamma.fit(df_gs_m.values, loc=0) # fit parameters of gamma distribution to SPI data 
params = [a, loc, scale]
rv = stats.gamma(*params) # Continuous random variable class, can sample randomly from the gamma distribution we just fitted  


fig = plt.figure(figsize=(30,10) )
l1=plt.plot(df_gs_m, '.k', markersize=20, color='b')
plt.ylabel('',size=15)
plt.title('',size=25)
plt.show()

### 3 - Intermediate step: Using Kolmogorov-Smirnov test to test goodness-of-fit:

In [ ]:
params = [a, loc, scale]
D, p = stats.kstest(df_gs_m, 'gamma', args=params)  
print('pvalue: ', p)

**Question 2**: Discuss whether the presented pvalue is sufficient. What hypothesis does the K-S test actually tests? 

In [ ]:
# df_gs_m is precipitation of single gridcel for a single month, with a single value for each year

fig = plt.figure(figsize=(30,10) )
# plot observed values
ax1 = plt.subplot(1, 2, 1)
vals = ax1.hist(df_gs_m, density=True, bins=20, label='Dist. Data')
# plot fitted distribution
x = np.linspace(stats.gamma.ppf(0.01, a, loc=loc, scale=scale),
                stats.gamma.ppf(0.99, a, loc=loc, scale=scale), 100)
ax1.plot(x[1:], rv.pdf(x)[1:], 'k-', lw=2, label='Gamma dist. fitted')
ax1.legend(loc='best')
ax1.set_title(f"K-S test pvalue: {round(p, 2)}", fontsize=25)

ax2 = plt.subplot(1, 2, 2)
ax2.plot(df_gs_m,'.k');
ax2.set_title('Raw data')

### 3 - step 3: Transform to standard normal distribution

In [ ]:
# 3c. Transform to standard normal distribution (and account for zero values (cfr.Stagge et al. 2015))  
indices_nonzero = np.nonzero(df_gs_m)[0]
nyears_zero = len(df_gs_m) - np.count_nonzero(df_gs_m)
ratio_zeros = nyears_zero / len(df_gs_m)

p_zero_mean = (nyears_zero + 1) / (2 * (len(df_gs_m) + 1))           

prob_gamma = (df_gs_m * 0 ) + p_zero_mean
# probability of values, given the Gamma distribution
prob_gamma[indices_nonzero] = ratio_zeros+((1-ratio_zeros)*rv.cdf(df_gs_m[indices_nonzero]))

# Transform 
prob_std = stats.norm.ppf(prob_gamma)                                   
prob_std[prob_std>3] = 3
prob_std[prob_std<-3] = -3 

In [ ]:
fig = plt.figure(figsize=(20,10) )

# plot observed values
ax1 = plt.subplot(1, 2, 1)
vals = ax1.plot(df_gs_m, prob_gamma, '.k');
ax1.set_ylabel('Cumulative probability')
ax1.set_xlabel('Aggregated Precipitation [mm]')


ax1 = plt.subplot(1, 2, 2)
vals = ax1.plot(prob_std, prob_gamma, '.k');
ax1.set_ylabel('Cumulative probability')
ax1.set_xlabel('SPI')


**Question 2:** Explain in words what information you can deduce from these two figures? For example, why is the Gamma distribution increasing sharply at low values compared to the standard normal distribution?   

**Question 3:** Explain in your own words why the conversion from gamma to standard normal distribution is desirable when quantifying drought.

# 4: Functions to calculate SPI 
What we have done in step 3, will now be written into a funciton which calculates SPI for a given gridcell on a specific day of year. Subsequently, we have written down a fuction that loops over all gridcells and days and store the results in an new xarray object.

In [ ]:
def calc_SPI_gs_doy(df_gs, doy):
    '''
    Calculates SPI for a given gridcell on a specific day of year
    '''
    # step 2: 
    # fit distribution
    df_gs_m = df_gs[df_gs.index.dayofyear == doy]
    # note floc avoids loc being fitted, see Stagge et al. 2015
    params = stats.gamma.fit(df_gs_m.values, loc=0) # fit parameters of gamma distribution to SPI data 
    rv = stats.gamma(*params) # Continuous random variable class, can sample randomly from the gamma distribution we just fitted  
    
    # Account for zero values (cfr.Stagge et al. 2015))  
    indices_nonzero = np.nonzero(df_gs_m)[0]
    nyears_zero = len(df_gs_m) - np.count_nonzero(df_gs_m)
    ratio_zeros = nyears_zero / len(df_gs_m)

    p_zero_mean = (nyears_zero + 1) / (2 * (len(df_gs_m) + 1))           
    prob_gamma = (df_gs_m * 0 ) + p_zero_mean
    # probability of values, given the Gamma distribution and excluding zeros
    prob_gamma[indices_nonzero] = ratio_zeros+((1-ratio_zeros)*rv.cdf(df_gs_m[indices_nonzero]))
    
    # Step 3:
    # Transform Gamma probabilities to standard normal probabilities
    prob_std = stats.norm.ppf(prob_gamma)                                   
    prob_std[prob_std>3] = 3
    prob_std[prob_std<-3] = -3 
    return prob_std

def calc_SPI_gs_pentad(df_gs, p):
    '''
    For a given dataframe, the centered single date for the pentad is taken and passed to calc_SPI_gs_doy(df_gs, doy).
    There 73 pentads of 5 in 365 days, hence p should be in range (0,73)
    '''
    i = np.arange(0,365, 5)[p]
    dates = df_gs.index
    idx_pentad = np.arange(i, dates.size, 365, dtype=int)
    doy = int(np.mean(dates[idx_pentad].dayofyear)) # central dayofyear for the given month {m}
    prob_std = calc_SPI_gs_doy(df_gs, doy)
    return prob_std

def calc_SPI_gs_month(df_gs, month):
    '''
    For a given dataframe,  the centered single date for the {month} is taken and passed to calc_SPI_gs_doy(df_gs, doy).
    There 73 pentads of 5 in 365 days, hence p should be in range (0,73)
    '''
    # to select the SPI for month m (with 1 = jan and 12 = Dec):
    m = month
    dates = df_gs.index
    doy = int(np.mean(dates.dayofyear[dates.month == m])) # central dayofyear for the given month {m}
    prob_std = calc_SPI_gs_doy(df_gs, doy)
    return prob_std



In [ ]:
def calc_SPI_from_daily(xarray, SPI_aggr, freq='months'):
    '''
    Function extracts the values of xarray that are not nans 
    and will pass them to calc_SPI_gs_. 
    One can choose to calculate SPI for all days, or for each month or pentad (5-day bins).
    Advisably, stick to monthly or pentads, for computational reasons.
    '''

    # Step 1: SPI monthly windowed aggregation
    months = [1, 2, 3, 6, 9, 12, 24]
    days = [31, 61, 91, 183, 274, 365, 730]
    SPI_to_days = {months[i]:days[i] for i in range(len(months))}
    SPI_window = SPI_to_days[SPI_aggr]
    dates = pd.to_datetime(xarray.time.values)
    n_yrs = np.unique(dates.year)

    # Finding only the non-NaN gridcells 
    np_pr = np.reshape(xarray.values, (dates.size, -1))
    n_gs_total = np_pr.shape[1]
    mask_NanIsFalse   = ~np.isnan(np_pr) # where are values not NaN 
    # Select only the non-NaN gridcells, and cast to dataframe.
    pr_no_nans  = np_pr[mask_NanIsFalse].reshape( dates.size, -1) 
    df_no_nans = pd.DataFrame(pr_no_nans, index=dates) # make a pandas dataframe
    # Calculate a sum of window size {SPI_aggr[months]}, using equal weights. 
    df_no_nans = df_no_nans.rolling(window=SPI_window, min_periods=1, center=True, axis=0).sum()
    n_gs_no_nans = int(df_no_nans.columns.size)

    # loop over gridcells cells and specific day of year
    for i, gs in enumerate(df_no_nans.columns):
        if freq == 'daily':
            new_dates = dates
            if i == 0:
                np_tofill = np.zeros( dates.dayofyear.size,  n_gs_no_nans)
            for doy in np.unique(dates.dayofyear):
                mask_doy = dates.dayofyear == doy
                np_tofill[mask_doy,i] = calc_SPI_gs_doy(df_no_nans[gs], doy)
                
        if freq == 'pentads':
            np_tofill = np.zeros( (int(dates.size/5),  n_gs_no_nans)) 
            new_dates = []
            for p, d in enumerate(range(0,365,5)):
                x = np.arange(d, dates.size, 365, dtype=int)
                new_dates.append(dates[x])
                if i == 0:
                    mask_NanIsFalse = mask_NanIsFalse[:new_dates.size]
                    idx_fill = np.arange(p, int(dates.size/5), 73)
                np_tofill[idx_fill,i] = calc_SPI_gs_pentad(df_no_nans[gs], p)
            new_dates = pd.to_datetime(sorted(np.concatenate(new_dates)))
            
        if freq == 'months':
            new_dates = xarray.resample(time='M', 
                                        label='left').mean().time.values 
            new_dates = pd.to_datetime(new_dates + pd.Timedelta('1D'))
            if i == 0:
                mask_NanIsFalse = mask_NanIsFalse[:new_dates.size]
                np_tofill = np.zeros( (new_dates.size,  n_gs_no_nans)) 
            for m in range(1,13):
                idx_fill = np.arange(m-1, new_dates.size, 12)
                np_tofill[idx_fill,i] = calc_SPI_gs_month(df_no_nans[gs], m)

            

    # return values in there original position
    np_time_space = np.zeros( (new_dates.size, n_gs_total) )
    np_time_space[mask_NanIsFalse] = np_tofill.ravel()
    np_latlon = np.reshape(np_time_space, (new_dates.size, 
                                           xarray.latitude.size, 
                                           xarray.longitude.size))

    # return xarray with same dimensions:
    dates = pd.to_datetime(new_dates)
    SPI_xr = xr.DataArray(np_latlon, coords=[dates, 
                                             xarray.latitude, xarray.longitude],
                            dims=['time', 'latitude', 'longitude'])
    mask_nans = mask_NanIsFalse[0].reshape(xarray.latitude.size, 
                                           xarray.longitude.size)
    mask = (('latitude', 'longitude'), mask_nans)
    SPI_xr['mask'] = mask
    SPI_xr.attrs['units'] = '[std]'
    return SPI_xr

# 5: Calculate SPI for Kenya gridpoints

In [ ]:
SPI_aggr = 6
freq = 'months' # can be ['months', 'pentads', 'daily']
SPI_xr = calc_SPI_from_daily(pr_kenya, SPI_aggr=SPI_aggr, freq=freq)

**Question 4:** Without any calculation, what will now be the climatological value of SPI. You can check calculating and plotting the climatology like was done above in Section 2a.

# 6: Calculate ENSO 3.4 index

Load Sea Surface Temperature (sst) data

In [ ]:
sst_Pac  = core_pp.import_ds_lazy(sst_Pacific, format_lon='only_east')

### 3a: Calculating a spatial mean
When calculating a spatial mean, one has to take into account that the earth is round, 
meaning that the equally spaced gridboxes do not represent equal surfaces of the earth. Admittendly, it will not matter much for countries close to the equator, but it is the neat thing to do. The function to take into account the differences in gridbox sizes is defined in the next cell.

In [ ]:
def area_weighted(xarray):
   # Area weighted, taking cos of latitude in radians     
    coslat = np.cos(np.deg2rad(xarray.coords['latitude'].values)).clip(0., 1.)
    area_weights = np.tile(coslat[..., np.newaxis],(1,xarray.longitude.size))
    area_weights = area_weights / area_weights.mean()
    return xr.DataArray(xarray.values * area_weights, coords=xarray.coords, 
                          dims=xarray.dims)

Like done before, we can easily select the dimensions over which we want to do a computation. Thus for calculating the spatial mean we simply type:

In [ ]:
sst_enso = sst_Pac.sel(latitude=slice(10,-10), longitude=slice(190,240))
# non-normalized timeseries
Enso_34_raw = area_weighted(sst_enso).mean(dim=('latitude','longitude'))

### 6b: Calculate SPI timeseries for Kenya

In [ ]:
SPI_Kenya_ts = area_weighted(SPI_xr).where(SPI_xr['mask']).mean(
                                            dim=('latitude','longitude'))

**Action**: In the cell below, standardize both the SPI and ENSO_34_raw timeseries.

In the cell below, we select the dates for which the SPI is calculated.

In [ ]:
Enso_34   = Enso_34_all.sel(time=SPI_Kenya_ts.time) 

# 7: Link between ENSO and Kenya drought?

## 7a: Visualizing timeseries

In [ ]:
# Figure 2

fig = plt.figure(figsize=(30,10) )
dates = pd.to_datetime(Enso_34.time.values)
l1=plt.plot(dates, Enso_34, color='b', label='ENSO',
           linewidth=3)
l2=plt.plot(dates, SPI_Kenya_ts, color='r', label='SPI_Kenya', alpha=0.6)

plt.legend(loc='best')
plt.ylabel('',size=15)
plt.title('',size=25)
plt.show()


**Action**: Copy-past the code to make figure 2 and add a smoothend timeseries of SPI by applying a rolling mean in the figure below. You should be able to make this work using examples from above.

## 7b: Cross-correlation matrix

Below, we calculate a cross-correlation matrix using pandas.

In [ ]:
data = np.concatenate([Enso_34.values[:,None], 
                       SPI_Kenya_ts.values[:,None], 
                       SPI_Kenya_ts_rm6.values[:,None]], axis=1)


df = pd.DataFrame(data, columns=['Enso_34', 'SPI_Kenya', 
                                 f"SPI_Kenya rm{rolling_mean_smooth}"],
            index=SPI_Kenya_ts['time'])
df.corr()

**Question** 6: What is your interpretation of the results of 7a and 7b? (insert figure)

Using stats.pearsonr(ts1,ts2) you can calculate the Pearson correlation coefficient and the concomitant p-values. 

**Question 7**: Find out if ENSO is indeed significantly influencing drought in Kenya? (insert screenshot of the code)

**Question 8**: What do you think that would happen if we aggregate SPI over very long or short time windows? Re-run the code starting from Step 5 and fill in 1 months and 24 months. What happends to the correlation coefficient (r) and the p-value? 